In [8]:
ReloadProject('deep_learning')

notebook_init.py imported and reloaded
forwarded symbol: Activation
forwarded symbol: Dense
forwarded symbol: Dict
forwarded symbol: InputLayer
forwarded symbol: List
forwarded symbol: Model
forwarded symbol: Sequential
forwarded symbol: Tuple
reloaded: gym
forwarded symbol: gym
reloaded: keras
forwarded symbol: keras
reloaded: model_optimization
forwarded symbol: model_optimization
reloaded: openai_wrapper
forwarded symbol: openai_wrapper
reloaded: policy_impl
forwarded symbol: policy_impl
reloaded: q_function_memoization
forwarded symbol: q_function_memoization
reloaded: q_learning
forwarded symbol: q_learning
reloaded: q_learning_impl
forwarded symbol: q_learning_impl
reloaded: q_learning_impl_v2
forwarded symbol: q_learning_impl_v2
reloaded: q_learning_v2
forwarded symbol: q_learning_v2


## Test the memoization container

In [6]:
container = q_function_memoization.TestContinuousMemoizationContainer()

expect 0.0: 0.0
expect 2.0: 2.0
expect 3.0: 3.0
expect 4.0: 4.0
expect 1: 1
expect 4.0: 4.0
expect 1: 1
expect 3.0: 3.0
expect 2: 2
expect 8.0: 8.0
expect 1: 1
expect 3.xxx (some average): 3.238405844044235
expect 5: 5


## Discrete Circular World

In [15]:
STATE_ZERO_ARRAY = np.zeros(1, dtype=int)
TARGET_STATE = 5


class CircularWorld(q_learning_v2.Environment):
    
    def __init__(self):
        super().__init__(state_array_size=1, action_space_size=3)

        self.debug_verbosity = 0
        
        # action encoding
        self._action_minus = 0
        self._action_stay = 1
        self._action_plus = 2

        
    #@ Override
    def TakeAction(self, action: q_learning_v2.Action) -> q_learning_v2.Reward:
        current_state = self.GetState()
        new_state = current_state
        reward = 0
        if action == self._action_plus:
            if current_state < TARGET_STATE:
                reward = 1.0
            else:
                reward = -1.0
            new_state = current_state + 1
            if new_state == 11:
                new_state = STATE_ZERO_ARRAY
        elif action == self._action_minus:
            if current_state > TARGET_STATE:
                reward = 1.0
            else:
                reward = -1.0
            new_state = current_state - 1
            if new_state == -1:
                new_state = STATE_ZERO_ARRAY + 10
        else:
            if current_state != TARGET_STATE:
                reward = -1.0
            else:
                reward = 1.0

        self._protected_SetState(new_state)
        if self.debug_verbosity >= 1:
            print('Action %s: (%s) -> (%s), reward: %s' % (
                action, current_state, new_state, reward))
        return reward

In [9]:
%%time

env = CircularWorld()
# Default distance_threshold is 0.1, making this function behaving pretty much like the standard memoization Q-function.
qfunc = q_function_memoization.ContinuousMemoizationQFunction(
    env, buffer_size=40, learning_rate=0.9, discount_factor=0.95)

# Train.
policy = policy_impl.RandomActionPolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=20, debug_verbosity=10)

GET: ([0], 0) -> 17.090272380032335
GET: ([0], 1) -> 17.43785711001324
GET: ([0], 2) -> 19.426594686887135
Action 2: ([0]) -> ([1]), reward: 1.0
GET: ([1], 0) -> 17.453524606235735
GET: ([1], 1) -> 17.36816999396912
GET: ([1], 2) -> 19.3980338105813
GET: ([0], 2) -> 19.426594686887135
SET: ([0], 2) <- 19.427978376735723
GET: ([1], 0) -> 17.453524606235735
GET: ([1], 1) -> 17.36816999396912
GET: ([1], 2) -> 19.3980338105813
Action 2: ([1]) -> ([2]), reward: 1.0
GET: ([2], 0) -> 17.426596516774982
GET: ([2], 1) -> 17.399242596721933
GET: ([2], 2) -> 19.392828294693814
GET: ([1], 2) -> 19.3980338105813
SET: ([1], 2) <- 19.42067157302134
GET: ([2], 0) -> 17.426596516774982
GET: ([2], 1) -> 17.399242596721933
GET: ([2], 2) -> 19.392828294693814
Action 2: ([2]) -> ([3]), reward: 1.0
GET: ([3], 0) -> 17.42079896441983
GET: ([3], 1) -> 17.39533927415827
GET: ([3], 2) -> 19.36407548232761
GET: ([2], 2) -> 19.392828294693814
SET: ([2], 2) <- 19.395567366859485
GET: ([3], 0) -> 17.42079896441983


In [10]:
%%time

env = CircularWorld()
qfunc = q_function_memoization.ContinuousMemoizationQFunction(
    env, buffer_size=40, learning_rate=0.9, discount_factor=0.95)
# Try a bigger distance_threshold.
qfunc.ChangeSettings(distance_threshold=1.0)

# Train.
policy = policy_impl.RandomActionPolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=20, debug_verbosity=10)

GET: ([3], 0) -> 16.679001653164978
GET: ([3], 1) -> 16.60945163858726
GET: ([3], 2) -> 18.58978456569902
Action 2: ([3]) -> ([4]), reward: 1.0
GET: ([4], 0) -> 16.655220082615728
GET: ([4], 1) -> 16.589701160623143
GET: ([4], 2) -> 18.521810800735402
GET: ([3], 2) -> 18.58978456569902
SET: ([3], 2) <- 18.59512669119867
GET: ([4], 0) -> 16.655220082615728
GET: ([4], 1) -> 16.589701160623143
GET: ([4], 2) -> 18.521810800735402
Action 2: ([4]) -> ([5]), reward: 1.0
GET: ([5], 0) -> 16.58877300356857
GET: ([5], 1) -> 18.451318985798057
GET: ([5], 2) -> 16.601003375992693
GET: ([4], 2) -> 18.521810800735402
SET: ([4], 2) <- 18.528058812930876
GET: ([5], 0) -> 16.58877300356857
GET: ([5], 1) -> 18.451318985798057
GET: ([5], 2) -> 16.601003375992693
Action 1: ([5]) -> ([5]), reward: 1.0
GET: ([5], 0) -> 16.58877300356857
GET: ([5], 1) -> 18.451318985798057
GET: ([5], 2) -> 16.601003375992693
GET: ([5], 1) -> 18.451318985798057
SET: ([5], 1) <- 18.521009631437142
GET: ([5], 0) -> 16.588773003

In [11]:
%%time

env = CircularWorld()
qfunc = q_function_memoization.MemoizationQFunction(
    env, learning_rate=0.9, discount_factor=0.95)

# Train.
policy = policy_impl.RandomActionPolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=20, debug_verbosity=10)

GET: ([10], 0) -> 19.04487407165037
GET: ([10], 1) -> 17.08520116238111
GET: ([10], 2) -> 17.574099247499454
Action 0: ([10]) -> ([9]), reward: 1.0
GET: ([9], 0) -> 18.99466043195549
GET: ([9], 1) -> 17.04187935654081
GET: ([9], 2) -> 17.09225684468922
GET: ([10], 0) -> 19.04487407165037
SET: ([10], 0) <- 19.04492207648698
GET: ([9], 0) -> 18.99466043195549
GET: ([9], 1) -> 17.04187935654081
GET: ([9], 2) -> 17.09225684468922
Action 0: ([9]) -> ([8]), reward: 1.0
GET: ([8], 0) -> 19.502783958323416
GET: ([8], 1) -> 16.994634751588528
GET: ([8], 2) -> 17.037306465235638
GET: ([9], 0) -> 18.99466043195549
SET: ([9], 0) <- 19.47434632756207
GET: ([8], 0) -> 19.502783958323416
GET: ([8], 1) -> 16.994634751588528
GET: ([8], 2) -> 17.037306465235638
Action 0: ([8]) -> ([7]), reward: 1.0
GET: ([7], 0) -> 19.543966059859407
GET: ([7], 1) -> 17.299057490150798
GET: ([7], 2) -> 16.994883309512595
GET: ([8], 0) -> 19.502783958323416
SET: ([8], 0) <- 19.560369377012137
GET: ([7], 0) -> 19.54396605

All works!

## Continous Circular World

In [6]:
STATE_ZERO_ARRAY = np.zeros(1, dtype=int)
TARGET_STATE = 5


class CircularWorldContinuousStates(q_learning_v2.Environment):
    
    def __init__(self):
        super().__init__(state_array_size=1, action_space_size=3)

        self.debug_verbosity = 0
        
        # action encoding
        self._action_minus = 0
        self._action_stay = 1
        self._action_plus = 2

        
    #@ Override
    def TakeAction(self, action: q_learning_v2.Action) -> q_learning_v2.Reward:
        current_state = self.GetState()
        new_state = current_state
        reward = 0
        
        rand_jump = np.random.rand()
        if action == self._action_plus:
            if current_state < TARGET_STATE - 0.25:
                reward = 1.0
            else:
                reward = -1.0
            new_state = current_state + rand_jump
            if new_state > 10.0:
                new_state = STATE_ZERO_ARRAY
        elif action == self._action_minus:
            if current_state > TARGET_STATE + 0.25:
                reward = 1.0
            else:
                reward = -1.0
            new_state = current_state - rand_jump
            if new_state < 0:
                new_state = STATE_ZERO_ARRAY + 10.0
        else:
            if not (TARGET_STATE - 0.25 <= current_state <= TARGET_STATE + 0.25):
                reward = -1.0
            else:
                reward = 1.0

        self._protected_SetState(new_state)
        if self.debug_verbosity >= 1:
            print('Action %s: (%s) -> (%s), reward: %s' % (
                action, current_state, new_state, reward))
        return reward

The normal memoization method would fail.

In [17]:
%%time

env = CircularWorldContinuousStates()
qfunc = q_function_memoization.MemoizationQFunction(
    env, learning_rate=0.9, discount_factor=0.95)

# Train.
policy = policy_impl.RandomActionPolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=20, debug_verbosity=10)

GET: ([0], 0) -> -0.050000000000000044
GET: ([0], 1) -> -0.050000000000000044
GET: ([0], 2) -> 1.0
Action 2: ([0]) -> ([0.83132917]), reward: 1.0
GET: ([0.83132917], 0) -> 0.0
GET: ([0.83132917], 1) -> 0.0
GET: ([0.83132917], 2) -> 0.0
GET: ([0], 2) -> 1.0
SET: ([0], 2) <- 1.0
GET: ([0.83132917], 0) -> 0.0
GET: ([0.83132917], 1) -> 0.0
GET: ([0.83132917], 2) -> 0.0
Action 0: ([0.83132917]) -> ([0.74493309]), reward: -1.0
GET: ([0.74493309], 0) -> 0.0
GET: ([0.74493309], 1) -> 0.0
GET: ([0.74493309], 2) -> 0.0
GET: ([0.83132917], 0) -> 0.0
SET: ([0.83132917], 0) <- -0.9
GET: ([0.74493309], 0) -> 0.0
GET: ([0.74493309], 1) -> 0.0
GET: ([0.74493309], 2) -> 0.0
Action 0: ([0.74493309]) -> ([0.03931432]), reward: -1.0
GET: ([0.03931432], 0) -> 0.0
GET: ([0.03931432], 1) -> 0.0
GET: ([0.03931432], 2) -> 0.0
GET: ([0.74493309], 0) -> 0.0
SET: ([0.74493309], 0) <- -0.9
GET: ([0.03931432], 0) -> 0.0
GET: ([0.03931432], 1) -> 0.0
GET: ([0.03931432], 2) -> 0.0
Action 0: ([0.03931432]) -> ([10.]),

Try the continuous memoization Q-function.

In [19]:
%%time

env = CircularWorldContinuousStates()
qfunc = q_function_memoization.ContinuousMemoizationQFunction(
    env, buffer_size=40, learning_rate=0.9, discount_factor=0.95)
# Try a bigger distance_threshold.
qfunc.ChangeSettings(distance_threshold=1.0)

# Train.
policy = policy_impl.RandomActionPolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=30, debug_verbosity=10)

GET: ([4.78292799], 0) -> 15.803182674855988
GET: ([4.78292799], 1) -> 13.13550956633792
GET: ([4.78292799], 2) -> 15.775077440536663
Action 0: ([4.78292799]) -> ([4.45998725]), reward: -1.0
GET: ([4.45998725], 0) -> 16.252481847204898
GET: ([4.45998725], 1) -> 13.13550956633792
GET: ([4.45998725], 2) -> 15.775077440536661
GET: ([4.78292799], 0) -> 15.803182674855988
SET: ([4.78292799], 0) <- 14.576190246845785
GET: ([4.45998725], 0) -> 14.576190246845785
GET: ([4.45998725], 1) -> 13.13550956633792
GET: ([4.45998725], 2) -> 15.775077440536661
Action 2: ([4.45998725]) -> ([5.25747046]), reward: 1.0
GET: ([5.25747046], 0) -> 14.576190246845785
GET: ([5.25747046], 1) -> 14.35429491400457
GET: ([5.25747046], 2) -> 15.747695909011982
GET: ([4.45998725], 2) -> 15.775077440536661
SET: ([4.45998725], 2) <- 15.94178774625891
GET: ([5.25747046], 0) -> 14.576190246845785
GET: ([5.25747046], 1) -> 14.35429491400457
GET: ([5.25747046], 2) -> 15.844741827635445
Action 2: ([5.25747046]) -> ([5.327740

Not bad at all!

Now try a different distance threashold parameter.

In [20]:
%%time

env = CircularWorldContinuousStates()
qfunc = q_function_memoization.ContinuousMemoizationQFunction(
    env, buffer_size=40, learning_rate=0.9, discount_factor=0.95)

# Train.
policy = policy_impl.RandomActionPolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=30, debug_verbosity=10)

GET: ([0.23396147], 0) -> 11.992212870612034
GET: ([0.23396147], 1) -> 11.910081465546673
GET: ([0.23396147], 2) -> 13.688380500032146
Action 2: ([0.23396147]) -> ([1.18326624]), reward: 1.0
GET: ([1.18326624], 0) -> 12.052042353024987
GET: ([1.18326624], 1) -> 11.904234714434162
GET: ([1.18326624], 2) -> 13.390994697957714
GET: ([0.23396147], 2) -> 13.688380500032146
SET: ([0.23396147], 2) <- 13.71813851675706
GET: ([1.18326624], 0) -> 12.052042353024987
GET: ([1.18326624], 1) -> 11.904234714434162
GET: ([1.18326624], 2) -> 13.390994697957714
Action 2: ([1.18326624]) -> ([1.51844833]), reward: 1.0
GET: ([1.51844833], 0) -> 10.607969973450304
GET: ([1.51844833], 1) -> 10.74213934422639
GET: ([1.51844833], 2) -> 13.482824272973197
GET: ([1.18326624], 2) -> 13.390994697957714
SET: ([1.18326624], 2) <- 13.766914223187854
GET: ([1.51844833], 0) -> 10.607969973450304
GET: ([1.51844833], 1) -> 10.74213934422639
GET: ([1.51844833], 2) -> 13.482824272973197
Action 2: ([1.51844833]) -> ([1.5950

Even more impressive!

Try the DL again here.

In [11]:
%%time

env = CircularWorldContinuousStates()
qfunc = q_learning_impl_v2.KerasModelQFunctionBatchWrite(
    env, (6, 20, 20), 1000, num_batch_write=2, learning_rate=0.9, discount_factor=0.95)
policy = policy_impl.MaxValueWithRandomnessPolicy(certainty = 0.9)

# Train.
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=40000, debug_verbosity=0)

# Test.
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=19, debug_verbosity=10)

GET: ([5.21166764], 0) -> [[0.4631435]]
GET: ([5.21166764], 1) -> [[0.4631435]]
GET: ([5.21166764], 2) -> [[0.4631435]]
Action 0: ([5.21166764]) -> ([4.9465534]), reward: -1.0
GET: ([4.9465534], 0) -> [[0.4631435]]
GET: ([4.9465534], 1) -> [[0.4631435]]
GET: ([4.9465534], 2) -> [[0.4631435]]
GET: ([5.21166764], 0) -> [[0.4631435]]
[PENDING] SET: ([5.21166764], 0) <- [[-0.45769793]]
GET: ([4.9465534], 0) -> [[0.4631435]]
GET: ([4.9465534], 1) -> [[0.4631435]]
GET: ([4.9465534], 2) -> [[0.4631435]]
Action 0: ([4.9465534]) -> ([4.09290418]), reward: -1.0
GET: ([4.09290418], 0) -> [[0.4631435]]
GET: ([4.09290418], 1) -> [[0.4631435]]
GET: ([4.09290418], 2) -> [[1.7930359]]
GET: ([4.9465534], 0) -> [[0.4631435]]
[PENDING] SET: ([4.9465534], 0) <- [[0.67936]]
GET: ([4.09290418], 0) -> [[0.4631435]]
GET: ([4.09290418], 1) -> [[0.4631435]]
GET: ([4.09290418], 2) -> [[1.7930359]]
Action 2: ([4.09290418]) -> ([4.66929353]), reward: 1.0
GET: ([4.66929353], 0) -> [[0.4631435]]
GET: ([4.66929353], 

In [14]:
%%time

env = CircularWorldContinuousStates()
qfunc = q_learning_impl_v2.KerasModelQFunctionBatchWrite(
    env, (6, 20, 20), 1000, num_batch_write=2, learning_rate=0.9, discount_factor=0.95)
policy = policy_impl.MaxValueWithRandomnessPolicy(certainty = 0.9)

# Train.
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=40000, debug_verbosity=0)

# Test.
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=19, debug_verbosity=10)

GET: ([4.39727421], 0) -> [[0.6872222]]
GET: ([4.39727421], 1) -> [[-0.20269132]]
GET: ([4.39727421], 2) -> [[1.2425072]]
Action 2: ([4.39727421]) -> ([5.25176828]), reward: 1.0
GET: ([5.25176828], 0) -> [[1.0246122]]
GET: ([5.25176828], 1) -> [[-0.42555207]]
GET: ([5.25176828], 2) -> [[-0.01661938]]
GET: ([4.39727421], 2) -> [[1.2425072]]
[PENDING] SET: ([4.39727421], 2) <- [[1.9002941]]
GET: ([5.25176828], 0) -> [[1.0246122]]
GET: ([5.25176828], 1) -> [[-0.42555207]]
GET: ([5.25176828], 2) -> [[-0.01661938]]
Action 0: ([5.25176828]) -> ([4.98809701]), reward: 1.0
GET: ([4.98809701], 0) -> [[0.9497026]]
GET: ([4.98809701], 1) -> [[-0.34517115]]
GET: ([4.98809701], 2) -> [[0.2144528]]
GET: ([5.25176828], 0) -> [[1.0246122]]
[PENDING] SET: ([5.25176828], 0) <- [[1.8144569]]
GET: ([4.98809701], 0) -> [[0.9497026]]
GET: ([4.98809701], 1) -> [[-0.34517115]]
GET: ([4.98809701], 2) -> [[0.2144528]]
Action 0: ([4.98809701]) -> ([4.32940692]), reward: -1.0
GET: ([4.32940692], 0) -> [[0.6485799

In [15]:
%%time

env = CircularWorldContinuousStates()
qfunc = q_learning_impl_v2.KerasModelQFunctionBatchWrite(
    env, (6, 20, 20), 1000, num_batch_write=2, learning_rate=0.9, discount_factor=0.95)
policy = policy_impl.MaxValueWithRandomnessPolicy(certainty = 0.9)

# Train.
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=40000, debug_verbosity=0)

# Test.
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=19, debug_verbosity=10)

GET: ([4.98735011], 0) -> [[0.46853876]]
GET: ([4.98735011], 1) -> [[0.58382785]]
GET: ([4.98735011], 2) -> [[2.008639]]
Action 2: ([4.98735011]) -> ([5.25659322]), reward: -1.0
GET: ([5.25659322], 0) -> [[0.6176461]]
GET: ([5.25659322], 1) -> [[0.5549115]]
GET: ([5.25659322], 2) -> [[1.7817668]]
GET: ([4.98735011], 2) -> [[2.008639]]
[PENDING] SET: ([4.98735011], 2) <- [[0.8242745]]
GET: ([5.25659322], 0) -> [[0.6176461]]
GET: ([5.25659322], 1) -> [[0.5549115]]
GET: ([5.25659322], 2) -> [[1.7817668]]
Action 2: ([5.25659322]) -> ([5.47948621]), reward: -1.0
GET: ([5.47948621], 0) -> [[0.74257433]]
GET: ([5.47948621], 1) -> [[0.5469476]]
GET: ([5.47948621], 2) -> [[1.393805]]
GET: ([5.25659322], 2) -> [[1.7817668]]
[PENDING] SET: ([5.25659322], 2) <- [[0.46987998]]
GET: ([5.47948621], 0) -> [[0.74257433]]
GET: ([5.47948621], 1) -> [[0.5469476]]
GET: ([5.47948621], 2) -> [[1.393805]]
Action 2: ([5.47948621]) -> ([5.98296929]), reward: -1.0
GET: ([5.98296929], 0) -> [[1.0025514]]
GET: ([5